<a href="https://colab.research.google.com/github/Muhammad-ali-aren/Attention-Is-All-You-Need-Pytorch-implementation/blob/main/The_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.  so first we make tokenizer.
2. we need to make the multihead attention mechanism.











In [57]:
import torch
import torch.nn as nn

In [58]:
#some varible have poor names, once its completely working i will change to have proper names
class MultiHeadAttention(nn.Module):
    def __init__(self,D=512,num_heads=8,mask=False,seq_len=5):
        super().__init__()
        assert D % num_heads == 0, 'model dims must be divisible by number of heads'
        self.head_dim = D // num_heads
        self.num_heads = num_heads
        self.mask = mask
        self.D = D
        self.scale =self.head_dim ** 0.5
        self.Q_linear = nn.Linear(D,D)
        self.K_linear = nn.Linear(D,D)
        self.V_linear = nn.Linear(D,D)
        self.linear_out = nn.Linear(D,D)
    def forward(self,x,out_encoder=None):
        B,N,L = x.shape
        Q = self.Q_linear(x)
        K = self.K_linear(x)
        V = self.V_linear(x)
        if torch.is_tensor(out_encoder):
            Q = self.Q_linear(x)
            K = self.K_linear(out_encoder)
            V = self.V_linear(out_encoder)
        # Batch, sequence_length, model_dims -> Batch, num_heads, seq_length, head_dim
        Q = Q.view(B,N,self.num_heads,self.head_dim).transpose(1,2)
        K = K.view(B,N,self.num_heads,self.head_dim).transpose(1,2)
        V = V.view(B,N,self.num_heads,self.head_dim).transpose(1,2)
        attention_score = torch.matmul(Q,K.transpose(-2,-1)) / self.scale
        if self.mask:
            mask = torch.tril(torch.ones(N,N))
            # seq_len x seq_len
            attention_score = attention_score.masked_fill(mask == 0,-1e9)
        probs = nn.functional.softmax(attention_score,dim=-1)
        out = torch.matmul(probs,V)
        # batch,num_heads,seq_length,head_dim -> batch, seq_length,num_heads,heads_dim
        out = out.transpose(1,2).contiguous().view(B,N,self.D)
        out = self.linear_out(out)
        return out


In [59]:
class EncoderBlock(nn.Module):
    def __init__(self,D=512,dff=2048,num_heads=8,mask=False,seq_len=5):
        super().__init__()
        self.MHA = MultiHeadAttention(D=D,num_heads=num_heads,mask=mask,seq_len=seq_len)
        self.LayerNorm1 = nn.LayerNorm(D)
        self.LayerNorm2 = nn.LayerNorm(D)
        self.FeedForward = nn.Sequential(
            nn.Linear(D,dff),
            nn.ReLU(),
            nn.Linear(dff,D)
        )
        self.dropout = nn.Dropout(0.1)
    def forward(self,x):
        out_mha = self.dropout(self.MHA(x))
        res1 = self.LayerNorm1(out_mha + x)
        out_ffd = self.dropout(self.FeedForward(res1))
        res2 = self.LayerNorm2(out_ffd + res1)
        return res2


In [60]:
class Encoder(nn.Module):
    def __init__(self,layers=6,D=512,dff=2048,num_heads=8,mask=False,seq_len=5):
        super().__init__()
        self.EncoderLayer = nn.ModuleList(EncoderBlock(D=D,num_heads=num_heads,mask=mask,seq_len=seq_len) for i in range(layers))
    def forward(self,x):
        for layer in self.EncoderLayer:
            x = layer(x)
        return x

In [61]:
class DecoderBlock(nn.Module):
    def __init__(self,D=512,ddf=2048,num_heads=8,mask=True,seq_len=5,input_feature_dim=512):
        super().__init__()
        self.MMHA = MultiHeadAttention(D=D,num_heads=num_heads,mask=mask,seq_len=seq_len)
        self.MHA = MultiHeadAttention(D=D,num_heads=num_heads,mask=False,seq_len=seq_len)
        self.LayerNorm1 = nn.LayerNorm(D)
        self.LayerNorm2 = nn.LayerNorm(D)
        self.LayerNorm3 = nn.LayerNorm(D)
        self.dropout = nn.Dropout(0.1)
        self.FeedForward = nn.Sequential(nn.Linear(D,ddf),
                                        nn.ReLU(),
                                        nn.Linear(ddf,D))
    def forward(self,x,out_encoder):
        out_mmha = self.dropout(self.MMHA(x)) #out_mmha = query and out_encoder = keys and values
        res1 = self.LayerNorm1(out_mmha + x)

        out_mha = self.MHA(res1,out_encoder)
        res2 = self.LayerNorm2(res1 + out_mha)
        out_FF = self.FeedForward(res2)
        out = self.LayerNorm3(res2 + out_FF)
        return out

In [62]:
class Decoder(nn.Module):
    def __init__(self,D=512,layers=6,ddf=2048,num_heads=8,mask=True,seq_len=5):
        super().__init__()
        self.DecoderLayers = nn.ModuleList(DecoderBlock(D=D,ddf=ddf,num_heads=num_heads,mask=mask,seq_len=seq_len) for _ in range(layers))
    def forward(self,x,out_encoder):
        for layer in self.DecoderLayers:
            x = layer(x,out_encoder)
        return x

In [63]:
class Transformer(nn.Module):
    def __init__(self,D=512,layers=6,dff=2048,num_heads=8,mask=True,seq_len=5):
        super().__init__()
        self.D = D
        self.layers = layers
        self.dff = dff
        self.num_heads = num_heads
        self.mask = mask
        self.seq_len = seq_len
        self.Embeddings = nn.Embedding(num_embeddings=1200,embedding_dim=512) #num_embedding is size of your vocabolary
        self.Encoder = Encoder(layers,D,dff,num_heads,seq_len)
        self.Decoder = Decoder(D,layers,dff,num_heads,mask,seq_len)
        self.linear_layer = nn.Linear(D,D)
    def forward(self,x):
        x_in = x[:,:-1]
        out_embed = self.Embeddings(x_in)
        out_encoder = self.Encoder(out_embed)
        # we will shift the input sentences to the right by one token
        out_embed_dec= self.Embeddings(x[:,1:])
        out_decoder = self.Decoder(out_embed_dec,out_encoder)
        out = nn.functional.softmax(self.linear_layer(out_decoder),dim=-1)
        return out

In [68]:
tokens = torch.randint(1,1200,(2,6))
print(tokens)
# out_encoder = None #torch.randn(2,5,512)
# print(tokens)

tensor([[ 819,  495,  395,  427,  875,  816],
        [1036, 1127,  976, 1149,   40, 1112]])


In [69]:
layers = 6

encoder = Transformer(layers=layers)
print(encoder(tokens).shape)

torch.Size([2, 5, 512])
